### RTM - Stress Test

A number of exploratory notebooks have validated the following RTM system.
- Entry is based on a 1 lot ATM option
  - With an up trend enter bull trade anywhere below the 21 period moving average
  - With a down trend enter bear trade anywhere above the 21 period moving average
  - With no trend enter enter bull trade in oversold condition and bear trade in overbought condition
- Exit is at a target consistent with a 0.5% move in the underlying or exit after 3 days
- Return is based on the value of stock controlled by a 1 lot of options.
- over the test period of 2007 to 2017 this system consistently shows an average annual return of 13% with almost no chance of drawdown over any 3 month period. The worst 1 month performance was better than -1%

This notebook will test the versatility of this system by testing the following:
- the effects of the system for different sector ETF's
- the effects of the system drawdown when combining uncorrelated sector ETF's

Assumptions:
- ATM Options cost approximately 3% of the underlying's stock price.
- Option Expiry is chosen to minimize Theta at about 1% of the option price. This is typically about 20 DTE.


In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

start_date = '2018-01-01'
end_date = '2018-04-01'

TARGET = 0.005
HOLD_TIME = 3
TARGET_DELTA = 0.55
HOLD_DELTA = 0.45
OPTION_PRICE = 0.03
COMMISSION = 6
THETA_FACTOR = 0.01
TRADE_LENGTH = 2

def RTMSysResults(symbols):
    data = get_pricing(symbols=symbols, start_date=start_date, end_date=end_date).copy()
    O = data['open_price']
    H = data['high']
    L = data['low']
    C = data['close_price']
    O1 = data['open_price'].shift(-1)

    # RETURN - Ten Day Return for Buy and Hold
    TenDayReturn = ((data['open_price'].shift(-10) - O1)/O1).dropna()

    # RETURN - Hold til target.
    max_rise10 = ((H[::-1].rolling(window=HOLD_TIME).max()[::-1]-O1)/O1).dropna()
    max_drop10 = ((L[::-1].rolling(window=HOLD_TIME).min()[::-1]-O1)/O1).dropna()

    BullTargetReturn = ((max_rise10 > TARGET)*TARGET*TARGET_DELTA + 
                    (max_rise10 < TARGET)*TenDayReturn*HOLD_DELTA).dropna().astype("float")
    BearTargetReturn = ((max_drop10 < -TARGET)*TARGET*TARGET_DELTA - 
                    (max_drop10 > -TARGET)*TenDayReturn*HOLD_DELTA).dropna().astype("float")
    BullTargetReturn[BullTargetReturn < -OPTION_PRICE] = -OPTION_PRICE
    BearTargetReturn[BearTargetReturn < -OPTION_PRICE] = -OPTION_PRICE

    #Calculate Overhead as a percentage
    Overhead = (COMMISSION/100)/O1 + OPTION_PRICE*THETA_FACTOR*TRADE_LENGTH
    
    # Identifying the Trend using moving average cross combinations.
    K = C.rolling(window=8).mean()
    mean = C.rolling(window=21).mean()
    slowMA = C.rolling(window=55).mean()
    bull_trend = (K > mean) & (mean > slowMA)
    bear_trend = (K < mean) & (mean < slowMA)
    no_trend = ~bull_trend & ~bear_trend

    data.loc[bull_trend,"trend"] = "Bull"
    data.loc[bear_trend,"trend"] = "Bear"
    data.loc[no_trend,"trend"] = "Neutral"

    # Price position relative to key moving averages.
    data.loc[(C > mean),"ma21"] = "AboveMA21"
    data.loc[(C < mean),"ma21"] = "BelowMA21"

    # Stochastic Status
    H14 = H.rolling(window=14).max().dropna()
    L14 = L.rolling(window=14).min().dropna()
    Stoch = 100*(C-L14)/(H14-L14)
    OB = Stoch > 80
    OS = Stoch < 20
    Neutral = ~OB & ~OS

    data.loc[OB,"stoch"] = "OverBought"
    data.loc[OS,"stoch"] = "OverSold"
    data.loc[Neutral,"stoch"] = "Neutral"

    # Determine System Results
    bullEntry = (data["stoch"]=="OverSold") & (data["trend"] == "Neutral")
    bearEntry = (data["stoch"]=="OverBought") & (data["trend"] == "Neutral")

    bullEntry = ((data["trend"]=="Bull") & (data["ma21"] == "BelowMA21")) | bullEntry 
    bearEntry = ((data["trend"]=="Bear") & (data["ma21"] == "AboveMA21")) | bearEntry

    df = (BullTargetReturn[bullEntry]-Overhead[bullEntry]).round(4)*100
    print("Bull trade Win Percentage: {0:.2f}%".format(100*float((df > 0).sum())/float(len(df.index))))
    bull = df.groupby(pd.Grouper(freq="M"))
    
    print(data[bullEntry])
    print(df)
    
    df = (BearTargetReturn[bearEntry]-Overhead[bearEntry]).round(4)*100
    print("Bear trade Win Percentage: {0:.2f}%".format(100*float((df > 0).sum())/float(len(df.index))))
    bear = df.groupby(pd.Grouper(freq="M"))
    
    print(df)
    
    df1 = pd.DataFrame({"Bull_Count":bull.count().round(0),"Bull_Sum":bull.sum(),
                   "Bear_Count":bear.count().round(0),"Bear_Sum":bear.sum()})
    df1 = df1.fillna(0)
    df1["Total_Count"] = df1["Bear_Count"] + df1["Bull_Count"]
    df1["Total_Sum"] = df1["Bear_Sum"] + df1["Bull_Sum"]
    df1.index = df1.index.strftime("%Y-%m")

    print("\n{} Monthly Results:\n=================================\n").format(symbols)
    plt.bar(pd.to_datetime(df1.index), df1["Total_Sum"])
    plt.show()

    Min_3Month = df1["Total_Sum"].rolling(window=3).sum().min()
    Max_3Month = df1["Total_Sum"].rolling(window=3).sum().max()
    Min_12Month = df1["Total_Sum"].rolling(window=12).sum().min()
    Max_12Month = df1["Total_Sum"].rolling(window=12).sum().max()

    #print(df1)
    print("\n Total Return: {0:.2f}% on {2:} trades       Average Annual Return: {1:.2f}% on {3:.2f} trades".
          format(df1.Total_Sum.sum(), df1.Total_Sum.mean()*12, df1.Total_Count.sum(), df1.Total_Count.mean()*12))
    print("\n3 Month Return: {0:.2f}% Min, {1:.2f}% Max".format(float(Min_3Month),float(Max_3Month)))
    print("12 Month Return: {0:.2f}% Min, {1:.2f}% Max".format(float(Min_12Month),float(Max_12Month)))
    
    return df1

In [ ]:
R = RTMSysResults('SPY')
results = pd.DataFrame(R.loc[:,'Total_Sum'])
results.columns = ['SPY']

In [ ]:
R = RTMSysResults('XLY')
results['XLY'] = R.Total_Sum

In [ ]:
R = RTMSysResults('XLE')
results['XLE'] = R.Total_Sum

In [ ]:
R = RTMSysResults('XLV')
results['XLV'] = R.Total_Sum

In [ ]:
R = RTMSysResults('XLK')
results['XLK'] = R.Total_Sum

In [ ]:
R = RTMSysResults('XLU')
results['XLU'] = R.Total_Sum

### Correlation

In [ ]:
results.corr()

### Mixed Results 
With some dramatic random draw downs in each individual set of results the idea with this section is to combine 5 of the least correlated sector results to minimize draw downs.

The percentages are based on the amount of underlying stock value 1 option lot controls.

In [ ]:
MixResults = results['XLY'] + \
            results['XLE'] + \
            results['XLV'] + \
            results['XLK'] + \
            results['XLU']
    
print("\nMixed Monthly Results for XLY,XLE,XLV,XLK,XLU :\n===============================================\n")
plt.bar(pd.to_datetime(MixResults.index), MixResults/5)
plt.show()

Min_3Month = MixResults.rolling(window=3).sum().min()/5
Max_3Month = MixResults.rolling(window=3).sum().max()/5
Min_12Month = MixResults.rolling(window=12).sum().min()/5
Max_12Month = MixResults.rolling(window=12).sum().max()/5

#print(df1)
print("\nTotal Return: {0:.2f}%      Average Annual Return: {1:.2f}%    Average Annual Return per Sector: {2:.2f}%".
      format(MixResults.sum(), MixResults.mean()*12, MixResults.mean()*12/5))
print("\nPer Sector Results:\n3 Month Return: {0:.2f}% Min, {1:.2f}% Max".format(float(Min_3Month),float(Max_3Month)))
print("12 Month Return: {0:.2f}% Min, {1:.2f}% Max".format(float(Min_12Month),float(Max_12Month)))


In [ ]:
M = MixResults.dropna()/5
print(M.describe())
print("\n Months above break even: {0:.2f}%".format(100*float((M > 0).sum())/float(len(M.index))))
plt.hist(M, bins=20);

This histogram shows the distribution of monthly returns from 2007 to 2017 using an equally weighted mix of 5 relatively uncorrelated sector ETF's. The maximum monthly loss with the mix was 6% and ~75% of the months were above break even. The average monthly return is 1.38%